In [1]:
from src.MilvusClientASOT import MilvusClientASOT
milvus_client = MilvusClientASOT()

/Users/tirsolopezausens/Desktop/aisot/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-04-13 23:16:28,345 - DEBUG - milvus_logger - Milvus Client successfully initialized.
2025-04-13 23:16:32,045 - DEBUG - milvus_logger - Dense embeddings initialized.


In [2]:
collection_name = 'asot'

In [3]:
milvus_client.list_episodes(collection_name)

2025-04-13 23:16:32,122 - DEBUG - milvus_logger - Found 9 unique episodes in collection asot


['1119', '190', '274', '313', '314', '322', '325', '329', '331']

In [5]:
# You can play with Milvus vector database here to check songs
test_query = """
glorified
"""

print("=" * 50)
print(f"\nQuerying with skills: {test_query}")
print("=" * 50)

results = milvus_client.sparse_search(
    collection_name=collection_name,
    query_text=test_query,
    limit=10,
)
print(f"Found {len(results)} matches:")
for i, hit in enumerate(results):
    print(f" {i}. Episode: {hit['entity']['episode_id']}")
    if hit['entity']['artist'] != 'nav':
        print(f" Artist: {hit['entity']['artist']}")
    if hit['entity']['collaborators'] != 'nav':
        print(f" Collaborators: {hit['entity']['collaborators']}")
    if hit['entity']['featured_artists'] != 'nav':
        print(f" Featured Artists: {hit['entity']['featured_artists']}")
    if hit['entity']['title'] != 'nav':
        print(f" Title: {hit['entity']['title']}")
    if hit['entity']['remix_info'] != 'nav':
        print(f" Remix: {hit['entity']['remix_info']}")
    if hit['entity']['ranking'] != -1:
        print(f" Ranking: {hit['entity']['ranking']}")
    if hit['entity']['popularity_score'] != -1:
        print(f" Popularity Score: {hit['entity']['popularity_score']}")
    if hit['entity']['vote_count'] != -1:
        print(f" Vote Count: {hit['entity']['vote_count']}")
    if hit['entity']['URL'] != 'nav':
        print(f" URL: {hit['entity']['URL']}")
    print(f" Match Score: {hit['distance']:.4f}")



Querying with skills: 
glorified

Found 1 matches:
 0. Episode: 322
 Artist: Graham Gold
 Title: Glorified
 Ranking: 18
 URL: https://www.astateoftrance.com/episode-322/
 Match Score: 5.4664
